## Allele filp QC YML generator
This module takes in a table of sumstat, with the columns: #chr, theme1, theme2, theme3 and each rows as 1 chr and the sumstat of corresponding chr and generate a list of yml to be used

In [ ]:
[global]
# List of path to the index of sumstat, each correspond to 1 recipe file documenting the path to the sumstat of each chromosome.
parameter: sumstat_list = paths
# List of names that corresponding to each of the studies
parameter: name = list
parameter: cwd = path
import pandas as pd
input_list = sumstat_list
sumstat_list = pd.read_csv(input_list[0],sep = "\t")
sumstat_list = sumstat_list.sort_values('#chr')
for x in range(1,len(input_list)):
    sumstat_list = sumstat_list.merge(pd.read_csv(input_list[x],sep = "\t"), on = "#chr")
sumstat_list.columns = ["#chr"] + name
sumstat_inv = sumstat_list.values.tolist()
names = "_".join(name)
## Assuming all the input sumstat are using the same header
parameter: CHR = "chrom"
parameter: POS = "pos"
parameter: A0 = "ref" 
parameter: A1 = "alt"
parameter: SNP = "variant_id"
parameter: STAT = "beta"
parameter: SE = "se"
parameter: P = "pval"
parameter: KEEP_AMBIGUOUS = "True"
parameter: GENE = "phenotype_id"
parameter: container = ""

In [ ]:
[yml_generator]
input: for_each  = "sumstat_inv"
output: f'{cwd:a}/{names}.{_sumstat_inv[0]}/{names}.{_sumstat_inv[0]}.yml'
python: expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout' , container = container
    header="INPUT:"
    body = [f'\n  - {x}:\n\
          ID: GENE,CHR,POS,A0,A1\n\
          CHR: $[CHR]\n\
          POS: $[POS]\n\
          A0: $[A0]\n\
          A1: $[A1]\n\
          SNP: $[SNP]\n\
          STAT: $[STAT]\n\
          SE: $[SE]\n\
          P: $[P]\n\
          GENE: $[GENE]\n\
          THEME: {y}' for x,y in zip($[_sumstat_inv[1:len(_sumstat_inv)]],$[name])]
    TARGET = "TARGET:" + body[0]
    OUTPUT = "OUTPUT: $[_output:d]"
    text = header+"\n".join(body)+"\n"+TARGET+"\n"+OUTPUT
    with open($[_output:r], 'w') as f:
        f.write(text)       

In [ ]:
[yml_list]
input: output_from("yml_generator"),group_by = "all"
output: f'{cwd:a}/yml_list.txt', f'{cwd:a}/qced_sumstat_list.txt'
import pandas as pd
yml_df = pd.DataFrame({"#chr" : sumstat_list["#chr"].values.tolist()  ,  "dir" : _input})
yml_df.to_csv(_output[0],sep = "\t",index = 0)
data_dir_tmp = pd.Series(_input)
data_dir = [f'{x:d}' for x in data_dir_tmp ]
theme = sumstat_list.columns.values.tolist()[1:]
for i in theme:
    sumstat_list = sumstat_list.assign(**{i : data_dir+ pd.Series([f'/{path(x):b}'  for x in  sumstat_list[i].values.tolist()])  } )
sumstat_list.to_csv(_output[1],sep = "\t", index = 0)